<b>Keresés random adatbázisban</b>

In [ ]:
from qiskit import QuantumCircuit,Aer,execute,transpile
from qiskit.algorithms import AmplificationProblem
import numpy as np
import matplotlib.pyplot as plt
import random
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor

In [ ]:
def oracle1(input):
    value = 6
    if input == value:
        return True
    else: return False

l = []
for i in range(1000):    

    randlist = random.sample(range(0,10), 10)

    for index, value in enumerate(randlist):
        if oracle1(value) == True:
            l.append(index+1)
            break
print("Átlagos futási szám:", np.mean(l))


<b>Keresés Grover algoritmussal</b>

In [ ]:
oracle = QuantumCircuit(2,name='oracle')
oracle.cz(0,1)
oracle.h(0)
oracle.to_gate()
oracle.draw(output='mpl')

In [ ]:
backend = Aer.get_backend('statevector_simulator')
grover_circ = QuantumCircuit(2,2)
grover_circ.h([1,1])
grover_circ.append(oracle,[0,1])
grover_circ.draw(output='mpl')

In [ ]:
job = execute(grover_circ,backend)
result=job.result()

In [ ]:
outputstate = result.get_statevector()
print(outputstate)


In [ ]:
reflection = QuantumCircuit(2,name = 'reflection')
reflection.h([0,1])
reflection.z([0,1])
reflection.cz(0,1)
reflection.h([0,1])
reflection.to_gate()
reflection.draw(output='mpl')

In [ ]:
backend = Aer.get_backend('qasm_simulator')
grover_circ=QuantumCircuit(2,2)
grover_circ.h([0,1])
grover_circ.append(oracle,[0,1])
grover_circ.append(reflection,[0,1])
grover_circ.measure([0,1],[0,1])
grover_circ.draw(output='mpl')

In [ ]:
transpiled_grover_circuit = transpile(grover_circ, backend, optimization_level=3)
job = backend.run(transpiled_grover_circuit)
job_monitor(job, interval=2)

job = execute(grover_circ, backend, shots = 1000)
result = job.result()
result.get_counts()

results = job.result()
answer = results.get_counts(grover_circ)
plot_histogram(answer)

<b>Szám faktorizációja Shor algoritmussal</b>

Qiskit implementation for Shor

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
print("Imports Successful")

In [ ]:
N = 15
a = 2

# Calculate the plotting data
xvals = np.arange(35)
yvals = [np.mod(a**x, N) for x in xvals]

# Use matplotlib to display it nicely
fig, ax = plt.subplots()
ax.plot(xvals, yvals, linewidth=1, linestyle='dotted', marker='x')
ax.set(xlabel='$x$', ylabel=f'${a}^x$ mod ${N}$',
       title="Example of Periodic Function in Shor's Algorithm")
try: # plot r on the graph
    r = yvals[1:].index(1) + 1
    plt.annotate('', xy=(0,1), xytext=(r,1),
                 arrowprops=dict(arrowstyle='<->'))
    plt.annotate(f'$r={r}$', xy=(r/3,1.5))
except ValueError:
    print('Could not find period, check a < N and have no common factors.')


In [ ]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)
    for _iteration in range(power):
        if a in [2,13]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [7,8]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    c_U = U.control()
    return c_U

In [ ]:
# Specify variables
N_COUNT = 8  # number of counting qubits
a =4

QFT circuit

In [ ]:
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT dagger"
    return qc

In [ ]:
# Create QuantumCircuit with N_COUNT counting qubits
# plus 4 qubits for U to act on
qc = QuantumCircuit(N_COUNT + 4, N_COUNT)

# Initialize counting qubits
# in state |+>
for q in range(N_COUNT):
    qc.h(q)

# And auxiliary register in state |1>
qc.x(N_COUNT)

# Do controlled-U operations
for q in range(N_COUNT):
    qc.append(c_amod15(a, 2**q),
             [q] + [i+N_COUNT for i in range(4)])

# Do inverse-QFT
qc.append(qft_dagger(N_COUNT), range(N_COUNT))

# Measure circuit
qc.measure(range(N_COUNT), range(N_COUNT))
qc.draw(fold=-1)  # -1 means 'do not fold'

In [ ]:
#Simulation
aer_sim = Aer.get_backend('aer_simulator')
t_qc = transpile(qc, aer_sim)
counts = aer_sim.run(t_qc).result().get_counts()
plot_histogram(counts)

Since we have 8 qubits, these results correspond to measured phases of:

In [ ]:
rows, measured_phases = [], []
for output in counts:
    decimal = int(output, 2)  # Convert (base 2) string to decimal
    phase = decimal/(2**N_COUNT)  # Find corresponding eigenvalue
    measured_phases.append(phase)
    # Add these values to the rows in our table:
    rows.append([f"{output}(bin) = {decimal:>3}(dec)",
                 f"{decimal}/{2**N_COUNT} = {phase:.2f}"])
# Print the rows in a table
headers=["Register Output", "Phase"]
df = pd.DataFrame(rows, columns=headers)
print(df)

The order (r) must be less than N, so we will set the maximum denominator to be 15:

In [ ]:
rows = []
for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append([phase,
                 f"{frac.numerator}/{frac.denominator}",
                 frac.denominator])
# Print as a table
headers=["Phase", "Fraction", "Guess for r"]
df = pd.DataFrame(rows, columns=headers)
print(df)

In [ ]:
def a2jmodN(a, j, N):
    """Compute a^{2^j} (mod N) by repeated squaring"""
    for _ in range(j):
        a = np.mod(a**2, N)
    return a
a2jmodN(7, 2049, 53)

Factoring 15

In [ ]:
N = 15
a = 7
print(a)

In [ ]:
gcd(a, N)

In [ ]:
def qpe_amod15(a):
    """Performs quantum phase estimation on the operation a*r mod 15.
    Args:
        a (int): This is 'a' in a*r mod 15
    Returns:
        float: Estimate of the phase
    """
    N_COUNT = 8
    qc = QuantumCircuit(4+N_COUNT, N_COUNT)
    for q in range(N_COUNT):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+N_COUNT) # And auxiliary register in state |1>
    for q in range(N_COUNT): # Do controlled-U operations
        qc.append(c_amod15(a, 2**q),
                 [q] + [i+N_COUNT for i in range(4)])
    qc.append(qft_dagger(N_COUNT), range(N_COUNT)) # Do inverse-QFT
    qc.measure(range(N_COUNT), range(N_COUNT))
    # Simulate Results
    aer_sim = Aer.get_backend('aer_simulator')
    # `memory=True` tells the backend to save each measurement in a list
    job = aer_sim.run(transpile(qc, aer_sim), shots=1, memory=True)
    readings = job.result().get_memory()
    print("Register Reading: " + readings[0])
    phase = int(readings[0],2)/(2**N_COUNT)
    print(f"Corresponding Phase: {phase}")
    return phase

In [ ]:
phase = qpe_amod15(a) # Phase = s/r
frac = Fraction(phase).limit_denominator(15)
s, r = frac.numerator, frac.denominator
print(r)

In [ ]:
guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
print(guesses)

In [ ]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)
    for _iteration in range(power):
        if a in [2,13]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [7,8]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    c_U = U.control()
    return c_U

In [ ]:
#Version for any N number
def c_amodN(a, power, N):
    """Controlled multiplication by a mod N"""
    if a < 0 or a >= N or power < 0:
        raise ValueError("'a' must be in the range [0, N) and 'power' must be a non-negative integer")

    num_qubits = N.bit_length()  # Determine the number of qubits needed based on N

    U = QuantumCircuit(num_qubits + 1)

    for _ in range(power):
        for q in range(num_qubits):
            U.swap(q, q + 1)

        for q in range(num_qubits):
            if (a << q) & (1 << num_qubits):
                U.x(q)

    U = U.to_gate()
    U.name = f"{a}^{power} mod {N}"
    c_U = U.control()
    return c_U

In [ ]:
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT dagger"
    return qc

In [ ]:
def qpe_amod15(a,N):
    """Performs quantum phase estimation on the operation a*r mod 15.
    Args:
        a (int): This is 'a' in a*r mod 15
    Returns:
        float: Estimate of the phase
    """
    N_COUNT = 8
    qc = QuantumCircuit(4+N_COUNT, N_COUNT)
    for q in range(N_COUNT):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+N_COUNT) # And auxiliary register in state |1>
    for q in range(N_COUNT): # Do controlled-U operations
        qc.append(c_amod15(a, 2**q),
                 [q] + [i+N_COUNT for i in range(4)])
    #qc.append(qft_dagger(N_COUNT), range(N_COUNT)) # Do inverse-QFT
    qc.append(QFT(N_COUNT).inverse(), range(N_COUNT))  # Do inverse-QFT
    qc.measure(range(N_COUNT), range(N_COUNT))
    # Simulate Results
    aer_sim = Aer.get_backend('aer_simulator')
    # `memory=True` tells the backend to save each measurement in a list
    job = aer_sim.run(transpile(qc, aer_sim), shots=100, memory=True)
    counts = job.result().get_counts()
    print(counts)
    readings = job.result().get_memory()
    print("Register Reading: " + readings[0])
    phase = int(readings[0],2)/(2**N_COUNT)
    print(f"Corresponding Phase: {phase}")
    return phase

Full Shor algo

In [ ]:
N=15
a = 7
FACTOR_FOUND = False
ATTEMPT = 0
while not FACTOR_FOUND:
    ATTEMPT += 1
    print(f"\nATTEMPT {ATTEMPT}:")
    phase = qpe_amod15(a,N) # Phase = s/r
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    print(f"Result: r = {r}")
    if phase != 0:
        # Guesses for factors are gcd(x^{r/2} ±1 , 15)
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        print(f"Guessed Factors: {guesses[0]} and {guesses[1]}")
        for guess in guesses:
            if guess not in [1,N] and (N % guess) == 0:
                # Guess is a factor!
                print(f'*** Non-trivial factor found: {guess} ***')
                FACTOR_FOUND = True

<b>RSA feltörése Shor algoritmussal</b>

In [ ]:
import RSA_Module
bit_length = int(input("Enter bit_length: "))

public, private = RSA_Module.generate_keypair(2**bit_length)
print(public[0],public[1])

<b>RSA átalakítás</b>

In [ ]:
msg = input("\nWrite message: ")
encrypted_msg, encryption_obj = RSA_Module.encrypt(msg, public)

print("\nEncrypted message: " + encrypted_msg)
print("\nEncrypted object: " , encryption_obj)

<b>RSA visszaalakítás</b>

In [ ]:
decrypted_msg = RSA_Module.decrypt(encryption_obj, private)

print("\nDecrypted message using RSA Algorithm: " + decrypted_msg)

<b>Shor algo</b>

In [ ]:
N_shor = public[1]
assert N_shor>0,"Input must be positive"
p,q = "Ide jönne a Shor vagy Grover algoritmus ami bármilyen N-re megmondja a faktorokat"
print(p,q)
phi = (p-1) * (q-1)  
d_shor = RSA_Module.mod_inverse(public[0], phi) # Privát kulcs kiszámítása

<b>Cracked RSA with Shor's algo</b>

In [ ]:
decrypted_msg = RSA_Module.decrypt(encryption_obj, (d_shor,N_shor))

print('\nMessage Cracked using Shors Algorithm: ' + decrypted_msg + "\n")